In [2]:
import pandas as pd

In [5]:
bike = pd.read_csv("./SeoulBikeData.csv", encoding = 'unicode_escape')
bike.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [7]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


## Write YAML file

In [21]:
%%writefile file.yaml
file_type: csv
dataset_name: bike
file_name: SeoulBikeData
table_name: 
inbound_delimiter: encoding = 'unicode_escape'
outbound_delimiter: "|"
skip_leading_rows: 0
columns: 
    - date
    - rented_bike_count
    - hour
    - temperature_c
    - humidity
    - wind_speed_m_s
    - visibility_10m
    - dew_point_temperature_c
    - solar_radiation_mj_m2
    - rainfall_mm
    - snowfall_cm
    - seasons
    - holiday
    - functioning_day

Overwriting file.yaml


In [22]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [23]:
config_data

{'file_type': 'csv',
 'dataset_name': 'bike',
 'file_name': 'SeoulBikeData',
 'table_name': None,
 'inbound_delimiter': "encoding = 'unicode_escape'",
 'outbound_delimiter': '|',
 'skip_leading_rows': 0,
 'columns': ['date',
  'rented_bike_count',
  'hour',
  'temperature_c',
  'humidity',
  'wind_speed_m_s',
  'visibility_10m',
  'dew_point_temperature_c',
  'solar_radiation_mj_m2',
  'rainfall_mm',
  'snowfall_cm',
  'seasons',
  'holiday',
  'functioning_day']}

In [24]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [25]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['date', 'rented_bike_count', 'hour', 'temperature_c', 'humidity',
       'wind_speed_m_s', 'visibility_10m', 'dew_point_temperature_c',
       'solar_radiation_mj_m2', 'rainfall_mm', 'snowfall_cm', 'seasons',
       'holiday', 'functioning_day'],
      dtype='object')
columns of YAML are: ['date', 'rented_bike_count', 'hour', 'temperature_c', 'humidity', 'wind_speed_m_s', 'visibility_10m', 'dew_point_temperature_c', 'solar_radiation_mj_m2', 'rainfall_mm', 'snowfall_cm', 'seasons', 'holiday', 'functioning_day']


In [26]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed
